# Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl

|składowa|selektor|zmienna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autor|user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommend|
|liczbę gwiazdek|span.user-post__score-count|stars|
|treść opinii|div.user-post__text|content|
|listę wad|review-feature__title--positives ~ div.review-feature__item|cons|
|listę zalet|review-feature__title--negatives ~ div.review-feature__item|pros|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|

## Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

## Wczytanie kodu produktu o którym pobierzemy opinie

In [ ]:
product_id = 104109736
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"


## Wysłanie do serwera Ceneo.pl żądania dostępu do strony z opiniami produktu

In [ ]:
response = requests.get(url)
response.status_code

## Przekształcenie kodu HTML w postachi tekstowej do obiektu reprezuntującego strukture DOM
## Wydobycie ze struktury DOM znaczników odpowiadająchc opiniom konsumentów

In [ ]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")
opinion = page.select_one("div.js_product-review")
print(type(page))
print(type(opinions))
print(type(opinion))

## Wydobycie ze struktury znacznika odpowiadającego pojedynczej opinii jej składowych

In [ ]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select("selector")]
        return [tag.text.strip() for tag in ancestor.select("selector")]
    if selector:
        if attribute:
            return ancestor.select_one(selector)[attribute].strip()
        return ancestor.select_one(selector).text.strip()
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

In [ ]:
single_option = {
    "opinion_id" : extract(opinion,None,"data-entry-id"),
    "author" : extract(opinion, "span.user-post__author-name"),
    "recommend" : extract(opinion, "span.user-post__author-recomendation > em"),
    "stars": extract(opinion, "span.user-post__score-count"),
    "content" : extract(opinion, "div.user-post__text"),
    "cons" : extract(opinion, "review-feature__title--positives ~ div.review-feature__item", None, True),
    "pros" : extract(opinion, "review-feature__title--negatives ~ div.review-feature__item", None, True),
    "opinion_date" : extract(opinion, "span.user-post__published > time:nth-child(1)","datetime"),
    "purchase_date" : extract(opinion, "span.user-post__published > time:nth-child(2)", "datetime"),
    "up_vote" : extract(opinion, "button.vote-yes", "data-total-vote"),
    'down_vote' : extract(opinion, "button.vote-no","data-total-vote")
}

In [ ]:
single_option